In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import logging
import os
import datetime

In [17]:
# 获取脚本的当前路径，避免计划执行时路径出错
home_dir = os.path.dirname(os.path.realpath(__file__))
# 更换workding directory
working_directory = home_dir
os.chdir(working_directory)

In [18]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [19]:
# 屏幕最大化，且指定下载目录
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

prefs = {"profile.default_content_settings.popups": 0,
         "directory_upgrade": True,
         "profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs", prefs)

In [20]:
# 使用chromedriver才可以用开发者权限
chrome_driver_path = ".//chrome_driver//chromedriver.exe" # 获取脚本的当前路径，避免计划执行时路径出错
# home_dir = os.path.dirname(os.path.realpath(__file__))
# # 更换workding directory
# working_directory = home_dir
# os.chdir(working_directory)r//chromedriver.exe"
# browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)

In [21]:
# main page
# main_page_url = "http://shopee.sg"

shop_info = [["https://shopee.co.id", 'tengus.id', 'tengus1803', 11184349, '+ Ikuti'],
             ['https://shopee.sg', 'tengus1.sg', 'tengus1803', 11918, '+ Follow'],
             ['https://shopee.ph', 'tengus.ph', 'tengus1803', 2215148, '+ Follow']]

# shop_info = [['https://shopee.sg', 'tengus1.sg', 'tengus1803', 11918, '+ Follow'],
#              ['https://shopee.ph', 'tengus.ph', 'tengus1803', 2215148, '+ Follow']]

# shop_info = [['https://shopee.ph', 'tengus.ph', 'tengus1803', 2215148, '+ Follow']]

# main_page_url = "http://shopee.co.id"
# browser.get(main_page_url)

# # remove ads, refresh again
# for i in range(5):
#     browser.get(main_page_url)
#     time.sleep(10)

In [22]:
# 重刷页面的脚本
def open_page(browser, page_url):
    i = 0
    while i == 0:
        try:
            # browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)
            browser.get(page_url)
            time.sleep(10)
            i = 1
        except:
            pass
        
# 刷新
def refresh_page(browser):
    i = 0
    while i == 0:
        try:
            # browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)
            browser.refresh()
            time.sleep(10)
            i = 1
        except:
            pass

In [23]:
for shop in shop_info:
    home_page = shop[0]
    shop_name = shop[1]
    shop_psw = shop[2]
    top_shop_id = shop[3]
    follow_button_text = shop[4]
    
    browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)
    open_page(browser, home_page)
    
    # remove ads, refresh again
    for i in range(5):
        refresh_page(browser)
        
    # login
    LoginElem = browser.find_elements_by_css_selector('.navbar__link.navbar__link--account.navbar__link--tappable.navbar__link--hoverable.navbar__link-text.navbar__link-text--medium')
    '''
    有关Selenium Compound class names not permitted的错误
    可以参考https://stackoverflow.com/questions/37771604/selenium-compound-class-names-not-permitted
    使用css_selector解决即可
    '''
    # 点击登入，弹出账户密码输入框
    LoginElem[1].click()
    time.sleep(10)
    
    acc_password_input_elem = browser.find_elements_by_css_selector('._3Ojta0._3H_lvW')
    try:
        acc_password_input_elem[0].click()
        acc_password_input_elem[0].send_keys(shop_name)
        acc_password_input_elem[1].click()
        acc_password_input_elem[1].send_keys(shop_psw)    
    except:
        time.sleep(30)
        acc_password_input_elem[0].click()
        acc_password_input_elem[0].send_keys(shop_name)
        acc_password_input_elem[1].click()
        acc_password_input_elem[1].send_keys(shop_psw)  

    time.sleep(10) 
    
    Login_button_elem = browser.find_elements_by_css_selector('._1BMmPI._37G57D._1qIIqG._3JP5il')
    time.sleep(10)
    Login_button_elem[0].click()
    time.sleep(10)    
    
    top_shop_url = home_page + '/shop/' + str(top_shop_id) + '/followers/?__classic__=1'
    open_page(browser, top_shop_url)
    
    # add certain number of fans
    to_add_num_of_following = 400
    num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
    following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
    # len(following_buttons)   
    
    i = 0
    total_add_num_of_following = to_add_num_of_following
    
    browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)
    
    while i <= total_add_num_of_following - 1:  
        while to_add_num_of_following > 0:
            if i + 1 > len(following_buttons):
                browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                time.sleep(10)

            following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
            logging.info('Now we have ' + str(len(following_buttons)) + ' following buttons on the screen.')
            logging.info('Now run the ' + str(i + 1) + ' time.')
            # if following_buttons[i].text == '+ 關注':
            # if following_buttons[i].text == '+ ติดตาม':
            # if following_buttons[i].text == '+ Follow':
            if following_buttons[i].text == follow_button_text:
                # print(following_buttons[i].text)
                shopid = following_buttons[i].get_attribute('shopid')
                following_buttons[i].click()
                time.sleep(5)
                to_add_num_of_following -= 1
                logging.info(str(shopid) + ' is following now, ' + str(to_add_num_of_following) + ' following remains.')
            else:
                logging.warning('Skip this one. It\'s following already.')
            i += 1    
            
    # 关闭
    browser.quit()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  
 2019-12-03 23:37:07,888 - INFO - Now we have 20 following buttons on the screen.
 2019-12-03 23:37:07,956 - INFO - Now run the 1 time.
 2019-12-03 23:37:13,241 - INFO - 31864896 is following now, 399 following remains.
 2019-12-03 23:37:13,253 - INFO - Now we have 20 following buttons on the screen.
 2019-12-03 23:37:13,254 - INFO - Now run the 2 time.
 2019-12-03 23:37:18,390 - INFO - 158030090 is following now, 398 following remains.
 2019-12-03 23:37:18,390 - INFO - Now we have 20 following buttons on the screen.
 2019-12-03 23:37:18,390 - INFO - Now run the 3 time.
 2019-12-03 23:37:23,515 - INFO - 153346984 is following now, 397 following remains.
 2019-12-03 23:37:23,515 - INFO - Now we have 20 following buttons on the screen.
 2019-12-03 23:37:23,531 - INFO - Now run the 4 time.
 2019-12-03 23:37:28,609 - INFO - 167542217 is following now, 396 followi

 2019-12-03 23:40:26,922 - INFO - 174104317 is following now, 363 following remains.
 2019-12-03 23:40:26,934 - INFO - Now we have 40 following buttons on the screen.
 2019-12-03 23:40:26,935 - INFO - Now run the 38 time.
 2019-12-03 23:40:32,062 - INFO - 199820140 is following now, 362 following remains.
 2019-12-03 23:40:32,074 - INFO - Now we have 40 following buttons on the screen.
 2019-12-03 23:40:32,078 - INFO - Now run the 39 time.
 2019-12-03 23:40:37,171 - INFO - 198848013 is following now, 361 following remains.
 2019-12-03 23:40:37,171 - INFO - Now we have 40 following buttons on the screen.
 2019-12-03 23:40:37,171 - INFO - Now run the 40 time.
 2019-12-03 23:40:42,265 - INFO - 205384940 is following now, 360 following remains.
 2019-12-03 23:40:52,328 - INFO - Now we have 60 following buttons on the screen.
 2019-12-03 23:40:52,328 - INFO - Now run the 41 time.
 2019-12-03 23:40:57,437 - INFO - 204616300 is following now, 359 following remains.
 2019-12-03 23:40:57,437 - 

 2019-12-03 23:43:58,515 - INFO - 195346901 is following now, 326 following remains.
 2019-12-03 23:43:58,515 - INFO - Now we have 80 following buttons on the screen.
 2019-12-03 23:43:58,531 - INFO - Now run the 75 time.
 2019-12-03 23:44:03,625 - INFO - 202297144 is following now, 325 following remains.
 2019-12-03 23:44:03,625 - INFO - Now we have 80 following buttons on the screen.
 2019-12-03 23:44:03,640 - INFO - Now run the 76 time.
 2019-12-03 23:44:08,734 - INFO - 198607816 is following now, 324 following remains.
 2019-12-03 23:44:08,734 - INFO - Now we have 80 following buttons on the screen.
 2019-12-03 23:44:08,734 - INFO - Now run the 77 time.
 2019-12-03 23:44:13,828 - INFO - 205487698 is following now, 323 following remains.
 2019-12-03 23:44:13,828 - INFO - Now we have 80 following buttons on the screen.
 2019-12-03 23:44:13,828 - INFO - Now run the 78 time.
 2019-12-03 23:44:18,922 - INFO - 192296176 is following now, 322 following remains.
 2019-12-03 23:44:18,922 - 

 2019-12-03 23:47:27,765 - INFO - 158619432 is following now, 289 following remains.
 2019-12-03 23:47:27,781 - INFO - Now we have 120 following buttons on the screen.
 2019-12-03 23:47:27,781 - INFO - Now run the 112 time.
 2019-12-03 23:47:32,859 - INFO - 125117406 is following now, 288 following remains.
 2019-12-03 23:47:32,859 - INFO - Now we have 120 following buttons on the screen.
 2019-12-03 23:47:32,859 - INFO - Now run the 113 time.
 2019-12-03 23:47:37,953 - INFO - 65377472 is following now, 287 following remains.
 2019-12-03 23:47:37,968 - INFO - Now we have 120 following buttons on the screen.
 2019-12-03 23:47:37,968 - INFO - Now run the 114 time.
 2019-12-03 23:47:43,047 - INFO - 90680569 is following now, 286 following remains.
 2019-12-03 23:47:43,062 - INFO - Now we have 120 following buttons on the screen.
 2019-12-03 23:47:43,062 - INFO - Now run the 115 time.
 2019-12-03 23:47:48,187 - INFO - 107721896 is following now, 285 following remains.
 2019-12-03 23:47:48,

 2019-12-03 23:50:51,875 - INFO - Now run the 148 time.
 2019-12-03 23:50:56,953 - INFO - 205315370 is following now, 252 following remains.
 2019-12-03 23:50:56,968 - INFO - Now we have 160 following buttons on the screen.
 2019-12-03 23:50:56,969 - INFO - Now run the 149 time.
 2019-12-03 23:51:02,047 - INFO - 172773594 is following now, 251 following remains.
 2019-12-03 23:51:02,065 - INFO - Now we have 160 following buttons on the screen.
 2019-12-03 23:51:02,066 - INFO - Now run the 150 time.
 2019-12-03 23:51:07,156 - INFO - 125376963 is following now, 250 following remains.
 2019-12-03 23:51:07,172 - INFO - Now we have 160 following buttons on the screen.
 2019-12-03 23:51:07,172 - INFO - Now run the 151 time.
 2019-12-03 23:51:12,281 - INFO - 139724427 is following now, 249 following remains.
 2019-12-03 23:51:12,297 - INFO - Now we have 160 following buttons on the screen.
 2019-12-03 23:51:12,297 - INFO - Now run the 152 time.
 2019-12-03 23:51:17,390 - INFO - 100931189 is f

...

 2019-12-03 23:57:51,156 - INFO - 200458628 is following now, 179 following remains.
 2019-12-03 23:57:51,172 - INFO - Now we have 240 following buttons on the screen.
 2019-12-03 23:57:51,172 - INFO - Now run the 222 time.
 2019-12-03 23:57:56,265 - INFO - 193450636 is following now, 178 following remains.
 2019-12-03 23:57:56,281 - INFO - Now we have 240 following buttons on the screen.
 2019-12-03 23:57:56,281 - INFO - Now run the 223 time.
 2019-12-03 23:58:01,375 - INFO - 142382046 is following now, 177 following remains.
 2019-12-03 23:58:01,390 - INFO - Now we have 240 following buttons on the screen.
 2019-12-03 23:58:01,390 - INFO - Now run the 224 time.
 2019-12-03 23:58:06,484 - INFO - 203595530 is following now, 176 following remains.
 2019-12-03 23:58:06,750 - INFO - Now we have 240 following buttons on the screen.
 2019-12-03 23:58:06,750 - INFO - Now run the 225 time.
 2019-12-03 23:58:11,843 - INFO - 202173201 is following now, 175 following remains.
 2019-12-03 23:58:1

 2019-12-04 00:01:13,380 - INFO - Now run the 258 time.
 2019-12-04 00:01:18,531 - INFO - 91099832 is following now, 142 following remains.
 2019-12-04 00:01:18,546 - INFO - Now we have 260 following buttons on the screen.
 2019-12-04 00:01:18,546 - INFO - Now run the 259 time.
 2019-12-04 00:01:23,656 - INFO - 95188446 is following now, 141 following remains.
 2019-12-04 00:01:23,687 - INFO - Now we have 260 following buttons on the screen.
 2019-12-04 00:01:23,687 - INFO - Now run the 260 time.
 2019-12-04 00:01:28,796 - INFO - 16267079 is following now, 140 following remains.
 2019-12-04 00:01:38,922 - INFO - Now we have 280 following buttons on the screen.
 2019-12-04 00:01:38,922 - INFO - Now run the 261 time.
 2019-12-04 00:01:44,031 - INFO - 197908043 is following now, 139 following remains.
 2019-12-04 00:01:44,046 - INFO - Now we have 280 following buttons on the screen.
 2019-12-04 00:01:44,046 - INFO - Now run the 262 time.
 2019-12-04 00:01:49,171 - INFO - 175355929 is foll

 2019-12-04 00:04:43,828 - INFO - Now run the 295 time.
 2019-12-04 00:04:48,937 - INFO - 33640399 is following now, 105 following remains.
 2019-12-04 00:04:48,968 - INFO - Now we have 300 following buttons on the screen.
 2019-12-04 00:04:48,968 - INFO - Now run the 296 time.
 2019-12-04 00:04:54,093 - INFO - 133792563 is following now, 104 following remains.
 2019-12-04 00:04:54,110 - INFO - Now we have 300 following buttons on the screen.
 2019-12-04 00:04:54,110 - INFO - Now run the 297 time.
 2019-12-04 00:04:59,234 - INFO - 67213646 is following now, 103 following remains.
 2019-12-04 00:04:59,250 - INFO - Now we have 300 following buttons on the screen.
 2019-12-04 00:04:59,250 - INFO - Now run the 298 time.
 2019-12-04 00:05:04,359 - INFO - 202980115 is following now, 102 following remains.
 2019-12-04 00:05:04,375 - INFO - Now we have 300 following buttons on the screen.
 2019-12-04 00:05:04,375 - INFO - Now run the 299 time.
 2019-12-04 00:05:09,500 - INFO - 101854720 is fol

 2019-12-04 00:08:17,143 - INFO - Now run the 332 time.
 2019-12-04 00:08:22,265 - INFO - 180897790 is following now, 68 following remains.
 2019-12-04 00:08:22,294 - INFO - Now we have 340 following buttons on the screen.
 2019-12-04 00:08:22,295 - INFO - Now run the 333 time.
 2019-12-04 00:08:27,392 - INFO - 133769307 is following now, 67 following remains.
 2019-12-04 00:08:27,423 - INFO - Now we have 340 following buttons on the screen.
 2019-12-04 00:08:27,424 - INFO - Now run the 334 time.
 2019-12-04 00:08:32,520 - INFO - 191228572 is following now, 66 following remains.
 2019-12-04 00:08:32,567 - INFO - Now we have 340 following buttons on the screen.
 2019-12-04 00:08:32,569 - INFO - Now run the 335 time.
 2019-12-04 00:08:37,685 - INFO - 65688817 is following now, 65 following remains.
 2019-12-04 00:08:37,713 - INFO - Now we have 340 following buttons on the screen.
 2019-12-04 00:08:37,714 - INFO - Now run the 336 time.
 2019-12-04 00:08:42,796 - INFO - 197839439 is follow

 2019-12-04 00:11:47,915 - INFO - Now run the 369 time.
 2019-12-04 00:11:53,056 - INFO - 202921100 is following now, 31 following remains.
 2019-12-04 00:11:53,090 - INFO - Now we have 380 following buttons on the screen.
 2019-12-04 00:11:53,091 - INFO - Now run the 370 time.
 2019-12-04 00:11:58,216 - INFO - 171981486 is following now, 30 following remains.
 2019-12-04 00:11:58,252 - INFO - Now we have 380 following buttons on the screen.
 2019-12-04 00:11:58,253 - INFO - Now run the 371 time.
 2019-12-04 00:12:03,363 - INFO - 203836379 is following now, 29 following remains.
 2019-12-04 00:12:03,395 - INFO - Now we have 380 following buttons on the screen.
 2019-12-04 00:12:03,396 - INFO - Now run the 372 time.
 2019-12-04 00:12:08,482 - INFO - 3555020 is following now, 28 following remains.
 2019-12-04 00:12:08,521 - INFO - Now we have 380 following buttons on the screen.
 2019-12-04 00:12:08,523 - INFO - Now run the 373 time.
 2019-12-04 00:12:13,611 - INFO - 200941651 is followi

In [24]:
# # login
# LoginElem = browser.find_elements_by_css_selector('.navbar__link.navbar__link--account.navbar__link--tappable.navbar__link--hoverable.navbar__link-text.navbar__link-text--medium')
# '''
# 有关Selenium Compound class names not permitted的错误
# 可以参考https://stackoverflow.com/questions/37771604/selenium-compound-class-names-not-permitted
# 使用css_selector解决即可
# '''
# # 点击登入，弹出账户密码输入框
# LoginElem[1].click()
# time.sleep(10)

In [25]:
# acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
# try:
# #     acc_password_input_elem[0].click()
# #     acc_password_input_elem[0].send_keys('tengus1.sg')
# #     acc_password_input_elem[1].click()
# #     acc_password_input_elem[1].send_keys('tengus1803')
    
#     acc_password_input_elem[0].click()
#     acc_password_input_elem[0].send_keys('tengus.id')
#     acc_password_input_elem[1].click()
#     acc_password_input_elem[1].send_keys('tengus1803')    
# except:
#     time.sleep(30)
# #     acc_password_input_elem = browser.find_elements_by_css_selector('._2QBp41._1b-IZR')
# #     acc_password_input_elem[0].click()
# #     acc_password_input_elem[0].send_keys('tengus1.sg')
# #     acc_password_input_elem[1].click()
# #     acc_password_input_elem[1].send_keys('tengus1803')

#     acc_password_input_elem[0].click()
#     acc_password_input_elem[0].send_keys('tengus.id')
#     acc_password_input_elem[1].click()
#     acc_password_input_elem[1].send_keys('tengus1803')  

# time.sleep(10)

In [26]:
# Login_button_elem = browser.find_elements_by_css_selector('._2DvX7K._3j9-lD._3ddytl.SjORHu')
# time.sleep(10)
# Login_button_elem[0].click()
# time.sleep(10)

In [27]:
# change current tab size
# browser.set_window_size(400, 862)    

In [28]:
# # add fans from Top Seller's shop
# # top_shop_id = 11918 # sg
# top_shop_id = 145423 # id

# # top_shop_url = 'https://shopee.sg/shop/' + str(top_shop_id) + '/followers/?__classic__=1'
# top_shop_url = 'https://shopee.co.id/shop/' + str(top_shop_id) + '/followers/?__classic__=1'
# browser.get(top_shop_url)

In [29]:
# # add certain number of fans
# to_add_num_of_following = 400
# num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))

In [30]:
# page down until we get at least 400 fans to cancel
# while num_of_following_display <= 1000:
    # body = browser.find_element_by_css_selector('body')
    # body.send_keys(Keys.PAGE_DOWN)
    # body.send_keys(Keys.END)
    # time.sleep(5)
    # num_of_following_display = len(browser.find_elements_by_css_selector('.clickable_area.middle-centered-div'))
    
# num_of_following_display

In [31]:
# following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
# # len(following_buttons)

In [32]:
# i = 0
# total_add_num_of_following = to_add_num_of_following

In [33]:
# browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.HOME)

In [34]:
# while i <= total_add_num_of_following - 1:  
#     while to_add_num_of_following > 0:
#         if i + 1 > len(following_buttons):
#             browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
#             time.sleep(10)
        
#         following_buttons = browser.find_elements_by_css_selector('.btn-follow.follow.L14')
#         logging.info('Now we have ' + str(len(following_buttons)) + ' following buttons on the screen.')
#         logging.info('Now run the ' + str(i + 1) + ' time.')
#         # if following_buttons[i].text == '+ 關注':
#         # if following_buttons[i].text == '+ ติดตาม':
#         # if following_buttons[i].text == '+ Follow':
#         if following_buttons[i].text == '+ Ikuti':
#             # print(following_buttons[i].text)
#             shopid = following_buttons[i].get_attribute('shopid')
#             following_buttons[i].click()
#             time.sleep(5)
#             to_add_num_of_following -= 1
#             logging.info(str(shopid) + ' is following now, ' + str(to_add_num_of_following) + ' following remains.')
#         else:
#             logging.warning('Skip this one. It\'s following already.')
#         i += 1

In [35]:
# # 关闭
# browser.quit()